In [ ]:
# https://iamtrask.github.io/2015/07/12/basic-python-network/

# my video walking thru this
X = np.array([[0,0,1], # 0
              [0,1,1], # 1
              [1,0,1], # 1
              [1,1,1]])# 0

y = np.array([[0,1,1,0]]).T
syn0 = 2*np.random.random((3,4)) - 1  # (3,4)
syn1 = 2*np.random.random((4,1)) - 1  #(4,1)
for j in range(6000):
    l1 = 1/(1+np.exp(-(np.dot(X,syn0)))) #(4,4)
    l2 = 1/(1+np.exp(-(np.dot(l1,syn1)))) #(4,1)
    l2_delta = (y - l2)*(l2*(1-l2))
    l1_delta = l2_delta.dot(syn1.T) * (l1 * (1-l1))
    syn1 += l1.T.dot(l2_delta)
    syn0 += X.T.dot(l1_delta)

In [ ]:
import numpy as np; np.set_printoptions(suppress=True)

def sig(x):
  return 1/(1 + np.exp(-x))

L1 = np.array([[-10, -5, -5,   5],
               [-10,  1, -5,   5],
               [  5, -1, 10, -10]])
L2 = np.array([[-1],[1],[1],[-1]])

H1 = sig(np.dot(X,L1))

#array([[0.99330715, 0.26894142, 0.9999546 , 0.0000454 ],    -1    -.99+.26+.99+0=.26  
#       [0.00669285, 0.5       , 0.99330715, 0.00669285],     1     0+.5+1+0 = 1.5
#       [0.00669285, 0.00247262, 0.99330715, 0.00669285],     1      1
#       [0.00000031, 0.00669285, 0.5       , 0.5       ]])   -1    0

# Note how L2 is used to manage H2 so that the values come out easy
# np.dot(A[0,:],L1[:,0]) 
H2 = sig(np.dot(H1,L2))

In [ ]:
import torch 
import numpy as np

X = torch.tensor([[0,0,1], 
                  [0,1,1], 
                  [1,0,1], 
                  [1,1,1]], dtype=torch.float)

y = torch.tensor([[0],[1],[1],[0]])

w = torch.tensor(np.ones((3,4)), dtype=torch.float, requires_grad=True)#.retain_grad()
w2 = torch.tensor(np.ones((4,1)), dtype=torch.float, requires_grad=True)#.retain_grad()

for _ in range(200):
  loss = torch.sigmoid(torch.matmul(torch.sigmoid(torch.matmul(X,w)),w2)-y)**2
  loss.backward()
  with torch.no_grad():
    w2 -= .01 * w2.grad
    w  -= .01 * w.grad

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

X = [[0, 0, 1], 
     [0, 1, 1], 
     [1, 0, 1], 
     [1, 1, 1]]

y = torch.tensor([[0],[1],[1],[0]])

class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = nn.Linear(3,4)
        self.linear2 = nn.Linear(4,4)
        self.linear3 = nn.Linear(4, 1)
    def forward(self, inputs):
        out = F.relu(self.linear1(inputs))
        out = self.linear2(out)
        out = self.linear3(out)
        log_probs = torch.sigmoid(out)
        return log_probs

losses = []
loss_function = nn.BCELoss()
model = SimpleModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)


for epoch in range(1000):
    total_loss = 0
    for idr, row in enumerate(X):
        model.zero_grad()
        log_probs = model(torch.tensor(row,dtype=torch.float))
        loss = loss_function(log_probs, torch.tensor(y[idr], dtype=torch.float).resize_((1)))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)

#check to see if it learned.


with torch.no_grad():
  model(torch.tensor(X[0],dtype=torch.float))
  model(torch.tensor(X[1],dtype=torch.float))
  model(torch.tensor(X[2],dtype=torch.float))
  model(torch.tensor(X[3],dtype=torch.float))
